# Accurate prediction of developer salaries from 2017 stackoverflow developer survey data

### Main Questions: Would the prediction result be different if I use get dummies function differently?

#### Q1. If you use the get dummie function according to the lecture method, the number of columns to increase is too large. Is there any way to reduce this?


#### Q2. Which column will give the best result?
Predict salary using each column and rank based on r2score


#### Q3. How many columns should be selected in order of ranking to obtain the best r2 score?
To get a better r2score, determine the number of columns, select the number of columns from the first place in the ranking, predict the salary, and get the r2score

#### Q4. What happens to the result when only the column with good results are selected?
Select a column from the first place in the ranking and check the result. If a bad result is found, the column is not selected and the next column is selected repeatedly.

#### Q5. Conclusion


### method 1 -> get dummies ordinary method
### method 2 -> get dummies my method(customized function)

# 1. Business Understanding
### The data I have collected is from Stackoverflow survey in 2017
### I want to investigate the annual income of developers based on this data

# 2. Data Understanding

## 2.1. Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

## 2.2 Import data

In [2]:
def import_data(data_path, drop_col=[]):
    df = pd.read_csv(data_path)
    print("Imported dataframe shape: ", df.shape)
    
    # drop specified columns
    df = df.drop(drop_col, axis=1)
    
    # drop column with all nan values
    df = df.dropna(how='all', axis=1)
    print(f"Df now has shape of {df.shape}")
    print(f"Importing df with shape of {df.shape} is complete!!!")
    return df

In [3]:
df = pd.read_csv('./survey_results_public_2017.csv')
print(f"df shape: {df.shape}")
df.head()

df shape: (51392, 154)


,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,...,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers,Salary,ExpectedSalary
0,1,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,...,Strongly disagree,Male,High school,White or of European descent,Strongly disagree,Strongly agree,Disagree,Strongly agree,NaN,NaN
1,2,Student,"Yes, both",United Kingdom,"Yes, full-time",Employed part-time,Some college/university study without earning ...,Computer science or software engineering,"More than half, but not all, the time",20 to 99 employees,...,Strongly disagree,Male,A master's degree,White or of European descent,Somewhat agree,Somewhat agree,Disagree,Strongly agree,NaN,37500.0
2,3,Professional developer,"Yes, both",United Kingdom,No,Employed full-time,Bachelor's degree,Computer science or software engineering,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A professional degree,White or of European descent,Somewhat agree,Agree,Disagree,Agree,113750.0,NaN
3,4,Professional non-developer who sometimes write...,"Yes, both",United States,No,Employed full-time,Doctoral degree,A non-computer-focused engineering discipline,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A doctoral degree,White or of European descent,Agree,Agree,Somewhat agree,Strongly agree,NaN,NaN
4,5,Professional developer,"Yes, I program as a hobby",Switzerland,No,Employed full-time,Master's degree,Computer science or software engineering,Never,10 to 19 employees,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2.3 Calculate ratio of nan values in Salary column

In [4]:
salary_nan_ratio = df.Salary.isnull().sum()/df.shape[0]
print("nan ratio of Salary column is ", salary_nan_ratio)
print(f"number of rows after drop: {df.dropna(axis=0, subset='Salary').shape[0]}")

nan ratio of Salary column is  0.7491632938978829
number of rows after drop: 12891


### Number of data available for predicting salary is 12891

## 2.4 Columns with multiple values in each cells and get dummies function
#### Cells of some columns, which are categorical datatypes, have multiple values and are separated by semi colons(;). In this case, when performing one hot encoding using pd.get_dummies function, a value of 1 should be given to all the corresponding columns, not just one column.

### Example of column with multiple values seperated by semi colons

In [5]:
print(df.HaveWorkedLanguage.shape)
df.HaveWorkedLanguage[:4]

(51392,)


0                            Swift
1    JavaScript; Python; Ruby; SQL
2                Java; PHP; Python
3           Matlab; Python; R; SQL
Name: HaveWorkedLanguage, dtype: object

### dummy will look like below, and it doesn't look good. And also dummy has 8438 columns

In [6]:
dummy = pd.get_dummies(df.HaveWorkedLanguage)
print(dummy.shape)
dummy.head(3)

(51392, 8438)


,Assembly,Assembly; C,Assembly; C#,Assembly; C#; Clojure,Assembly; C#; CoffeeScript; Dart; Go; Haskell; Java; JavaScript; Lua; Matlab; PHP; Python; R; Ruby; Rust; Smalltalk; SQL; TypeScript,Assembly; C#; CoffeeScript; JavaScript; SQL; Swift; TypeScript,Assembly; C#; Common Lisp; Elixir; F#,Assembly; C#; Common Lisp; Elixir; Groovy; R; Rust; Smalltalk; VB.NET; Visual Basic 6,Assembly; C#; Common Lisp; JavaScript; PHP; Python; SQL; VBA,Assembly; C#; F#; Java; JavaScript; Objective-C; PHP; Python; SQL; Swift; VB.NET; VBA,...,Swift; TypeScript,Swift; VB.NET,TypeScript,VB.NET,VB.NET; VBA,VB.NET; VBA; Visual Basic 6,VB.NET; Visual Basic 6,VBA,VBA; Visual Basic 6,Visual Basic 6
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### It seems that number of columns in dummy is too big. I think If the number of columns can be reduced, better results can be obtained.

# 3. Data Preparation
### Each data of the preprocessed columns are stored in the form of a dictionary  for various trials

## 3.1. Use Salary column as y(label), so if the value of the salary column is nan, row has to be dropped

In [7]:
print(df.shape)
df = df.dropna(subset='Salary', axis=0)
df.shape

(51392, 154)


(12891, 154)

In [8]:
y = df.Salary
X = df.drop('Salary', axis=1)
print(y.shape)
print(X.shape)

(12891,)
(12891, 153)


## 3.2. Remove unnecessary columns or columns with all values of nan

In [9]:
print(X.shape)
X = X.drop('Respondent', axis=1)
print(df.shape)
X = X.dropna(how='all', axis=1)
print(X.shape)

(12891, 153)
(12891, 154)
(12891, 145)


## 3.3. If the data type of the column is numeric, fill in the nan value with the average value.

In [10]:
def get_numeric_type_columns(df):
    df_num = df.select_dtypes(include=['int', 'float'])
    df_num_cols = df_num.columns
    print("Numeric columns from this df: ", df_num.shape[1])
    return df_num_cols

In [11]:
def fill_mean(series):
    # Mean function
    mean = series.mean()
    # Fill the mean
    series = series.fillna(mean, axis=0)
    return series

In [12]:
def get_numeric_dict(df):
    df_num_cols = get_numeric_type_columns(df)
    numeric_df_dict = {}
    numeric_df_names_list = []
    for col in df_num_cols:
        df_filled = fill_mean(df[col])
        numeric_df_dict[col] = df_filled
        numeric_df_names_list.append(col)
    return numeric_df_dict, numeric_df_names_list

### I will save each columns result in dictionary for later use

In [13]:
numeric_dict, numeric_df_names_list = get_numeric_dict(X)
numeric_df_names_list

Numeric columns from this df:  4


['CareerSatisfaction',
 'JobSatisfaction',
 'HoursPerWeek',
 'StackOverflowSatisfaction']

## 3.4. If the data type of the column is object, get dummies
### The part of finding the dummy will be written in detail in chapter 4 - Q1

# 4. Modeling
### In this section I will find solutions to my questions

### Main Questions: Would the prediction result be different if I use get dummies func differently?

#### Q1. If you use the get dummie function according to the lecture method, the number of columns to increase is too large. Is there any way to reduce this?


#### Q2. Which column will give the best result?
Predict salary using each column and rank based on r2score


#### Q3. How many columns should be selected in order of ranking to obtain the best r2 score?
To get a better r2score, determine the number of columns, select the number of columns from the first place in the ranking, predict the salary, and get the r2score

#### Q4. What happens to the result when only the column with good results are selected?
Select a column from the first place in the ranking and check the result. If a bad result is found, the column is not selected and the next column is selected repeatedly.

#### Q5. Conclusion

## Q1. If you use the get dummie function according to the lecture method, the number of columns to increase is too large. Is there any way to reduce this?
#### method 1 -> get dummies ordinary method
#### method 2 -> get dummies my method(customized function)

### Q1.1. Define functions to get dummies

In [14]:
def get_dummy_simple(series, dummy_na=False):
    df = pd.DataFrame()
    try:
        df = pd.get_dummies(series, prefix=series.name, prefix_sep='_', drop_first=False, dummy_na=dummy_na)
    except:
        print(f"Error with get_dummy_simple(), column: {series.name}")
    return df

In [15]:
def get_object_type_columns(df):
    df_obj = df.select_dtypes(include=['object'])
    df_obj_cols = df_obj.columns
    print("Object columns from this df: ", df_obj.shape[1])
    return df_obj_cols

In [16]:
def get_multiple_value_columns(df):
    contain_semicolons = df.apply(lambda col: 
                                  col.str.contains(';').any()
                                  if col.dtypes==object
                                  else False)
    df_contain_semicolons = df.loc[:, contain_semicolons]
    multiple_value_columns = df_contain_semicolons.columns
    return multiple_value_columns

In [17]:
def categorize_feature(series):
    series = series.apply(lambda value: list(value.split(';'))
                       if type(value)!=float
                       else float('nan'))
    return pd.get_dummies(series.apply(pd.Series).stack(dropna=False), prefix=series.name, prefix_sep='_').sum(level=0)    

In [18]:
def get_dummy_dict(df, multiple_value_columns=[]):
    df_obj_cols = get_object_type_columns(df)
    categorized_df_dict = {}
    categorized_df_names_list = []
    for col in df_obj_cols:
        if col in multiple_value_columns:
            df_dummy = categorize_feature(df[col]) # my method
        else:
            df_dummy = get_dummy_simple(df[col]) # simple method
        categorized_df_dict[col] = df_dummy
        categorized_df_names_list.append(col)
    return categorized_df_dict, categorized_df_names_list

### Q1.2. Store dummies in dictionaries

In [19]:
multiple_value_columns = get_multiple_value_columns(X)
multiple_value_columns

Index(['DeveloperType', 'MobileDeveloperType', 'ImportantBenefits',
       'JobProfile', 'EducationTypes', 'SelfTaughtTypes', 'CousinEducation',
       'HaveWorkedLanguage', 'WantWorkLanguage', 'HaveWorkedFramework',
       'WantWorkFramework', 'HaveWorkedDatabase', 'WantWorkDatabase',
       'HaveWorkedPlatform', 'WantWorkPlatform', 'IDE', 'Methodology',
       'MetricAssess', 'StackOverflowDevices', 'Gender', 'Race'],
      dtype='object')

#### Q1.2.1. Method 1 dictionary - get dummies ordinary method

In [20]:
dummy_dict1, _ = get_dummy_dict(X)

Object columns from this df:  141


#### Q1.2.2. Method 2 dictionary - get dummies my method(customized function)

In [21]:
dummy_dict2, _ = get_dummy_dict(X, multiple_value_columns)

Object columns from this df:  141


#### Q1.2.3. Checking that the dictionary was created correctly

In [22]:
assert X.shape[1] == len(numeric_dict) + len(dummy_dict1)
assert X.shape[1] == len(numeric_dict) + len(dummy_dict2)

### Q1.3. compare dummy(method 1 vs method 2)

In [23]:
col = 'HaveWorkedLanguage'

#### Q1.3.1. method 1

In [24]:
print(dummy_dict1[col].shape)
dummy_simple = dummy_dict1[col]
dummy_simple.head(2)

(12891, 3445)


,HaveWorkedLanguage_Assembly,HaveWorkedLanguage_Assembly; C,HaveWorkedLanguage_Assembly; C#,HaveWorkedLanguage_Assembly; C#; CoffeeScript; JavaScript; SQL; Swift; TypeScript,HaveWorkedLanguage_Assembly; C#; Common Lisp; JavaScript; PHP; Python; SQL; VBA,HaveWorkedLanguage_Assembly; C#; Go; Hack; Haskell; JavaScript; Lua; Objective-C; Perl; PHP; Python; Ruby; Rust; SQL; Swift; TypeScript; VBA,HaveWorkedLanguage_Assembly; C#; Go; Java; JavaScript; PHP; SQL,HaveWorkedLanguage_Assembly; C#; Go; JavaScript; PHP; Ruby; SQL,HaveWorkedLanguage_Assembly; C#; Groovy; Java; JavaScript; Matlab; Objective-C; Python; R; Swift,HaveWorkedLanguage_Assembly; C#; Java; JavaScript; Lua; Objective-C; PHP; SQL,...,HaveWorkedLanguage_Scala; SQL; TypeScript,HaveWorkedLanguage_Scala; SQL; VB.NET; VBA,HaveWorkedLanguage_Smalltalk,HaveWorkedLanguage_Smalltalk; SQL,HaveWorkedLanguage_Smalltalk; TypeScript,HaveWorkedLanguage_Swift,HaveWorkedLanguage_Swift; TypeScript,HaveWorkedLanguage_VB.NET,HaveWorkedLanguage_VBA,HaveWorkedLanguage_Visual Basic 6
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Q1.3.2. method 2

In [25]:
print(dummy_dict2[col].shape)
dummy_my = dummy_dict2[col]
dummy_my.head(2)

(12891, 68)


,HaveWorkedLanguage_ C,HaveWorkedLanguage_ C#,HaveWorkedLanguage_ C++,HaveWorkedLanguage_ Clojure,HaveWorkedLanguage_ CoffeeScript,HaveWorkedLanguage_ Common Lisp,HaveWorkedLanguage_ Dart,HaveWorkedLanguage_ Elixir,HaveWorkedLanguage_ Erlang,HaveWorkedLanguage_ F#,...,HaveWorkedLanguage_R,HaveWorkedLanguage_Ruby,HaveWorkedLanguage_Rust,HaveWorkedLanguage_SQL,HaveWorkedLanguage_Scala,HaveWorkedLanguage_Smalltalk,HaveWorkedLanguage_Swift,HaveWorkedLanguage_VB.NET,HaveWorkedLanguage_VBA,HaveWorkedLanguage_Visual Basic 6
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Q1.3.3. Method 2 has fewer columns and multiple columns can be selected

##### Q1.3.3.1 Number of selected columns in method 1

In [26]:
number_of_ones = dummy_simple.loc[14, :].sum()
print(f"Number of selected columns in dummy1 is {number_of_ones}")

Number of selected columns in dummy1 is 1


##### Q1.3.3.2. Number of selected columns in method 2

In [27]:
number_of_ones = dummy_my.loc[14, :].sum()
print(f"Number of selected columns in dummy2 is {number_of_ones}")

Number of selected columns in dummy2 is 3


## Q2. Which column will give the best result?

Predict salary using each column and rank based on r2score

### Q2.1. Define functions to get r2scores

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import time

result_summary = [] # [[condition, rank_test, rank_train, time consumption], ...]

In [29]:
def train_model(X, y):
    lm_model = LinearRegression(normalize=True) # Instantiate
    lm_model.fit(X, y) #Fit
    return lm_model

In [30]:
def check_result(col, df, y):
#     print(col)
#     print(f"shape: {df.shape}")
    
    X = df      
    #Split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=42) 
    
    model = train_model(X_train, y_train)

    #Predict and score the model
    y_test_preds = model.predict(X_test) 
    y_train_preds = model.predict(X_train)
    test_score = r2_score(y_test, y_test_preds)
    train_score = r2_score(y_train, y_train_preds)
#     print(f"test_score: {test_score}, train_score: {train_score}")
#     print('-'*40)
    
    return test_score, train_score

In [31]:
def get_rank_each_column(dict_num, dict_dummy):
    rank_test = []
    rank_train = []

    for col, df_num in dict_num.items():
        test_score, train_score = check_result(col, df_num.values.reshape(-1,1), y)

        rank_test.append([test_score, col])
        rank_train.append([train_score, col])

    for col, df_obj in dict_dummy.items():   
        test_score, train_score = check_result(col, df_obj, y)

        rank_test.append([test_score, col])
        rank_train.append([train_score, col])
    return rank_test, rank_train

### Q2.2. Get prediction from each column

#### Q2.2.1. Prediction results for each columns(method 1)

In [32]:
start_time = time.time()
rank_test, rank_train = get_rank_each_column(numeric_dict, dummy_dict1)
time_consumption = time.time()-start_time

In [33]:
rank_test.sort(reverse=True)
print("r2 score", "            ", "Column name")
rank_test[:3]

r2 score              Column name


[[0.4167738538485196, 'Currency'],
 [0.20301667135459334, 'YearsCodedJob'],
 [0.17681542205634249, 'YearsProgram']]

In [34]:
print(f"Best single column to predict the salary is {rank_test[0][1]}")
"Time consumption: ", time_consumption

Best single column to predict the salary is Currency


('Time consumption: ', 56.01122832298279)

##### Add result to summary

In [35]:
# [condition, rank_test, rank_train, time consumption]
result_summary.append([["method 1", "single column"], rank_test, rank_train, time_consumption]) 

#### Q2.2.2. Prediction results for each columns(method 2)

In [36]:
start_time = time.time()
rank_test, rank_train = get_rank_each_column(numeric_dict, dummy_dict2)
time_consumption = time.time() - start_time

In [37]:
rank_test.sort(reverse=True)
print("r2 score", "            ", "Column name")
rank_test[:3]

r2 score              Column name


[[0.4167738538485196, 'Currency'],
 [0.20301667135459334, 'YearsCodedJob'],
 [0.17681542205634249, 'YearsProgram']]

In [38]:
print(f"Best single column to predict the salary is {rank_test[0][1]}")
"Time consumption: ", time_consumption

Best single column to predict the salary is Currency


('Time consumption: ', 0.9449629783630371)

##### Add result to summary

In [39]:
# [condition, rank_test, rank_train, time consumption]
result_summary.append([["method 2", "single column"], rank_test, rank_train, time_consumption]) 

## Q3. How many columns should be selected in order of ranking to obtain the best r2 score?
#### To get a better r2score, determine the number of columns, select the number of columns from the first place in the ranking, predict the salary, and get the r2score

### Q3.1. Define number of features and function to ranking of r2scores

In [40]:
num_features = [1, 2, 3, 4, 5, 6, 7, 10, 20, 50, 100, X.shape[1]-3, X.shape[1]-2, X.shape[1]-1, X.shape[1]]

In [41]:
def get_rank_number_column(dict_num, dict_dummy, rank):
    rank_test, rank_train = [], []
    for num in num_features:
        features = []
        for i in range(num):
            features.append(rank[i][1])
        X = pd.DataFrame()
        for feature in features:
            if feature in dict_num:
                X = pd.concat([X, dict_num[feature]], axis=1)
            elif feature in dict_dummy:
                X = pd.concat([X, dict_dummy[feature]], axis=1)
            else:
                print(f"{feature} not in numeric_dict or categorized_df_dict!!!")
                

        test_score, train_score = check_result(num, X, y)
        rank_test.append([test_score, num])
        rank_train.append([train_score, num])
        
        print(f"number of features: {num}, r2score: {test_score}, shape of X: {X.shape}")
    return rank_test, rank_train

### Q3.2. Get r2 scores using method 1

In [42]:
rank = result_summary[0][1]
start_time = time.time()
rank_test, rank_train = get_rank_number_column(numeric_dict, dummy_dict1, rank)
time_consumption = time.time()-start_time

number of features: 1, r2score: 0.4167738538485196
number of features: 2, r2score: 0.5533608651193768
number of features: 3, r2score: 0.5579702570388851
number of features: 4, r2score: 0.5907981786641967
number of features: 5, r2score: -2.8933110678915737e+24
number of features: 6, r2score: -3.89664570061889e+23
number of features: 7, r2score: -3.342687532038344e+24
number of features: 10, r2score: -1.3860895637801805e+25
number of features: 20, r2score: -6.550066498594873e+23
number of features: 50, r2score: -2.447397800897439e+26
number of features: 100, r2score: -4.3798824048001777e+24
number of features: 142, r2score: -1.159010143378907e+28
number of features: 143, r2score: -292.51430030864503
number of features: 144, r2score: -1.9209987489382225
number of features: 145, r2score: -0.42987120036592863


In [43]:
rank_test.sort(reverse=True)
print("r2 score", "            ", "Number of Columns")
rank_test[:5]

r2 score              Number of Columns


[[0.5907981786641967, 4],
 [0.5579702570388851, 3],
 [0.5533608651193768, 2],
 [0.4167738538485196, 1],
 [-0.42987120036592863, 145]]

In [44]:
print(f"Best number of columns to predict the salary is {rank_test[0][1]}")
"Time consumption: ", time_consumption

Best number of columns to predict the salary is 4


('Time consumption: ', 1053.7508380413055)

In [45]:
# [condition, rank_test, rank_train, time consumption]
result_summary.append([["method 1", "number of columns"], rank_test, rank_train, time_consumption]) 

### Q3.3. Get r2 scores using method 2

In [46]:
rank = result_summary[1][1]
start_time = time.time()
rank_test, rank_train_number2 = get_rank_number_column(numeric_dict, dummy_dict2, rank)
time_consumption = time.time()-start_time

number of features: 1, r2score: 0.4167738538485196
number of features: 2, r2score: 0.5533608651193768
number of features: 3, r2score: 0.5579702570388851
number of features: 4, r2score: 0.5907981786641967
number of features: 5, r2score: 0.6057809978127665
number of features: 6, r2score: 0.6094893088177588
number of features: 7, r2score: -5.430516754220984e+23
number of features: 10, r2score: -1.4121282631674564e+26
number of features: 20, r2score: -3.7539335231166577e+25
number of features: 50, r2score: -3.979132219365716e+24
number of features: 100, r2score: -9.608937855305889e+23
number of features: 142, r2score: -1.8441373746018724e+24
number of features: 143, r2score: -5.126914621982429e+23
number of features: 144, r2score: -5.845549151340118e+23
number of features: 145, r2score: -1.9138531130635174e+24


In [47]:
rank_test.sort(reverse=True)
print("r2 score", "            ", "Number of Columns")
rank_test[:5]

r2 score              Number of Columns


[[0.6094893088177588, 6],
 [0.6057809978127665, 5],
 [0.5907981786641967, 4],
 [0.5579702570388851, 3],
 [0.5533608651193768, 2]]

In [48]:
print(f"Best number of columns to predict the salary is {rank_test[0][1]}")
"Time consumption: ", time_consumption

Best number of columns to predict the salary is 6


('Time consumption: ', 8.216017723083496)

In [49]:
# [condition, rank_test, rank_train, time consumption]
result_summary.append([["method 2", "number of columns"], rank_test, rank_train, time_consumption])

## Q4. What happens to the result when only the column with good results are selected?
#### Select a column from the first place in the ranking and check the result. If a bad result is found, the column is not selected and the next column is selected repeatedly.

### Q4.1.  Get rank, define function that gives best result

In [50]:
import numpy as np

rank_method1 = np.array(result_summary[0][1])[:, 1]
rank_method2 = np.array(result_summary[1][1])[:, 1]

In [51]:
def get_cols_gives_best_results(dict_num, dict_dummy, ranks):
    best_cols = []
    features = []
    best_score = 0

    for rank in ranks:
        features.append(rank)

        X = pd.DataFrame()
        for feature in features:
            if feature in dict_num:
                X = pd.concat([X, dict_num[feature]], axis=1)
            elif feature in dict_dummy:
                X = pd.concat([X, dict_dummy[feature]], axis=1)
            else:
                print(f"{feature} not in numeric_dict or categorized_df_dict!!!")
        test_score, train_score = check_result(len(features), X, y)

        if test_score > best_score:
            best_cols.append(rank)
            best_score = test_score
        else:
            features.pop(-1)

    print(f"Best score: {round(best_score, 3)}")
    print(f"Number of features: {len(features)}")
#     print(f"Best features: {best_cols}")

### Q4.2 Method 1 results

In [52]:
start_time = time.time()
get_cols_gives_best_results(numeric_dict, dummy_dict1, rank_method1)
print(f"Time Consumption: {time.time()-start_time}")

Best score: 0.659
Number of features: 27
Time Consumption: 85.64120817184448


In [53]:
# [condition, rank_test, rank_train, time consumption]
result_summary.append([["method 1", "columns with best results"], rank_test, rank_train, time_consumption])

### Q4.3. Method 2 results

In [54]:
start_time = time.time()
get_cols_gives_best_results(numeric_dict, dummy_dict2, rank_method2)
print(f"Time Consumption: {time.time()-start_time}")

Best score: 0.62
Number of features: 8
Time Consumption: 10.781554222106934


In [55]:
# [condition, rank_test, rank_train, time consumption]
result_summary.append([["method 2", "columns with best results"], rank_test, rank_train, time_consumption])

## Q5. Conclusion

#### There was no significant difference in the r2 score of Method 1 and Method 2.
#### A big difference was found in the time consumption
#### The time consumed for method 2 was found to be less than 1/60 to 1/6 compared to method 1


### Q5.1. Summary

In [58]:
condition, rank_test, _, time_consumption = result_summary[0]
print(f"Condition(dummy method): {condition[0]}")
print(f"Condition(number of columns): {condition[1]}")
print(f"The column to predict the best r2score: {rank_test[0][1]}, r2score: {round(rank_test[0][0], 3)}")
print(f"Time consumption: {round(time_consumption, 1)} seconds")

Condition(dummy method): method 1
Condition(number of columns): single column
The column to predict the best r2score: Currency, r2score: 0.417
Time consumption: 56.0 seconds


In [59]:
condition, rank_test, _, time_consumption = result_summary[1]
print(f"Condition(dummy method): {condition[0]}")
print(f"Condition(number of columns): {condition[1]}")
print(f"The column to predict the best r2score: {rank_test[0][1]}, r2score: {round(rank_test[0][0], 3)}")
print(f"Time consumption: {round(time_consumption, 1)} seconds")

Condition(dummy method): method 2
Condition(number of columns): single column
The column to predict the best r2score: Currency, r2score: 0.417
Time consumption: 0.9 seconds


In [60]:
condition, rank_test, _, time_consumption = result_summary[2]
print(f"Condition(dummy method): {condition[0]}")
print(f"Condition(number of columns): {condition[1]}")
print(f"The number of columns to predict the best r2score: {rank_test[0][1]}, r2score: {round(rank_test[0][0], 3)}")
print(f"Time consumption: {round(time_consumption, 1)} seconds")

Condition(dummy method): method 2
Condition(number of columns): number of columns
The number of columns to predict the best r2score: 4, r2score: 0.591
Time consumption: 1053.8 seconds


In [61]:
condition, rank_test, _, time_consumption = result_summary[3]
print(f"Condition(dummy method): {condition[0]}")
print(f"Condition(number of columns): {condition[1]}")
print(f"The number of columns to predict the best r2score: {rank_test[0][1]}, r2score: {round(rank_test[0][0], 3)}")
print(f"Time consumption: {round(time_consumption, 1)} seconds")

Condition(dummy method): method 2
Condition(number of columns): number of columns
The number of columns to predict the best r2score: 6, r2score: 0.609
Time consumption: 8.2 seconds


### Q5.2. Time consumption comparison

#### Q5.2.1 Time consumption to predict salary using each columns

In [62]:
# single column regression time consumption
time1 = result_summary[0][3]
time2 = result_summary[1][3]
print(f"Method 1: {time1}, Method 2: {time2}, Method 2/Method 1: {time2/time1}")

Method 1: 56.01122832298279, Method 2: 0.9449629783630371, Method 2/Method 1: 0.016870956175322718


#### Q5.2.2 Time consumption to predict salary using more than 1 column

In [63]:
# multiple columns regression time consumption
time3 = result_summary[2][3]
time4 = result_summary[3][3]
print(f"Method 1: {time3}, Method 2: {time4}, Method 2/Method 1: {time4/time3}")

Method 1: 1053.7508380413055, Method 2: 8.216017723083496, Method 2/Method 1: 0.0077969263951953695


#### Q5.2.3 Time consumption to get best results

In [64]:
# time consumption when finding best result 
time3 = result_summary[2][3]
time4 = result_summary[3][3]
print(f"Method 1: {time3}, Method 2: {time4}, Method 2/Method 1: {time4/time3}")

Method 1: 1053.7508380413055, Method 2: 8.216017723083496, Method 2/Method 1: 0.0077969263951953695


### Q5.3. Conclusion

### I compared two methods of finding a dummy.

### Both methods showed no significant difference in r2score.

### However, since the time required for method 2 was significantly smaller than that of method 1, it seems reasonable to choose method 2